In [9]:
import pandas as pd
import numpy as np
import joblib

In [29]:
df = pd.read_csv(r'C:\Users\tothm\PycharmProjects\Projet 7\dashboard\app_train_dashboard.csv')

data = df.copy()

In [3]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,141617,0,Cash loans,F,N,Y,0,135000.0,808650.0,23773.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,263512,0,Cash loans,M,N,Y,0,135000.0,254700.0,14751.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,2.0
2,397087,0,Revolving loans,F,Y,Y,0,234000.0,720000.0,36000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,0.0
3,316730,1,Cash loans,F,N,N,2,67500.0,545040.0,25407.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,441436,0,Cash loans,F,N,Y,1,225000.0,817560.0,29493.0,...,0,0,0,0,0.0,1.0,1.0,0.0,0.0,0.0


In [12]:
with open('../model_weights/clf_xgb_o.pkl', 'rb') as f:
    model = joblib.load(f)

with open('../model_weights/imputer.pkl', 'rb') as f:
    imputer = joblib.load(f)

with open('../model_weights/colonnes_attendues.pkl', 'rb') as f:
    colonnes_model = joblib.load(f)

with open('../model_weights/scaler.pkl', 'rb') as f:
    scaler = joblib.load(f)

C:\Users\tothm\.virtualenvs\Projet_7-_BujKIZF\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.2.0 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\tothm\.virtualenvs\Projet_7-_BujKIZF\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.0 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
# Prépare la requête pour qu'elle soit conforme au modèle
# ONE HOT ENCODING
data = pd.get_dummies(data)
print("Étape 3 réussie.")

# VALEURS ABERRANTES
# Create an anomalous flag column
data['DAYS_EMPLOYED_ANOM'] = data["DAYS_EMPLOYED"] == 365243
# Replace the anomalous values with nan
data['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)
print("Étape 4 réussie.")

# Traitement des valeurs négatives
data['DAYS_BIRTH'] = abs(data['DAYS_BIRTH'])
print("Étape 5 réussie.")

# CREATION DE VARIABLES
# CREDIT_INCOME_PERCENT: the percentage of the credit amount relative to a client's income
# ANNUITY_INCOME_PERCENT: the percentage of the loan annuity relative to a client's income
# CREDIT_TERM: the length of the payment in months (since the annuity is the monthly amount due
# DAYS_EMPLOYED_PERCENT: the percentage of the days employed relative to the client's age

data['CREDIT_INCOME_PERCENT'] = data['AMT_CREDIT'] / data['AMT_INCOME_TOTAL']
data['ANNUITY_INCOME_PERCENT'] = data['AMT_ANNUITY'] / data['AMT_INCOME_TOTAL']
data['CREDIT_TERM'] = data['AMT_ANNUITY'] / data['AMT_CREDIT']
data['DAYS_EMPLOYED_PERCENT'] = data['DAYS_EMPLOYED'] / data['DAYS_BIRTH']
print("Étape 6 réussie.")

# Récupère les colonnes attendues par le modèle
colonnes_attendues = colonnes_model
print(colonnes_attendues)
print("Étape 7 réussie.")

# Identify the columns that are missing from the received dataframe
missing_columns = set(colonnes_attendues) - set(data.columns)
print("Étape 8 réussie.")

# Add the missing columns to the received dataframe with a default value
for col in missing_columns:
    data[col] = 0
print("Étape 9 réussie.")

# Reorder the columns to match the order of the expected columns
data = data[colonnes_attendues]
data = data.reindex(columns=colonnes_attendues)
print("Étape 10 réussie.")

# Transform the data using the imputer and scaler
data = data[imputer.feature_names_in_]
data = imputer.transform(data)
print("Étape 11 réussie.")
data = scaler.transform(data)
print("Étape 12 réussie.")

# Make a prediction using the model
prediction = model.predict(data)
prediction_proba = model.predict_proba(data)
print("Étape 13 réussie.")

Étape 3 réussie.
Étape 4 réussie.
Étape 5 réussie.
Étape 6 réussie.
Index(['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       ...
       'WALLSMATERIAL_MODE_Panel', 'WALLSMATERIAL_MODE_Stone, brick',
       'WALLSMATERIAL_MODE_Wooden', 'EMERGENCYSTATE_MODE_No',
       'EMERGENCYSTATE_MODE_Yes', 'DAYS_EMPLOYED_ANOM',
       'CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM',
       'DAYS_EMPLOYED_PERCENT'],
      dtype='object', length=243)
Étape 7 réussie.
Étape 8 réussie.
Étape 9 réussie.
Étape 10 réussie.
Étape 11 réussie.
Étape 12 réussie.
Étape 13 réussie.


In [14]:
prediction_proba

array([[0.56948066, 0.43051937],
       [0.8947431 , 0.10525689],
       [0.7770156 , 0.22298437],
       ...,
       [0.785899  , 0.21410102],
       [0.64967835, 0.35032165],
       [0.9940809 , 0.00591907]], dtype=float32)

In [16]:
results = pd.DataFrame(prediction_proba, columns=["Prediction : 0", "Prediction : 1"])

In [30]:
df = pd.concat([df, results], axis=1)

In [31]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,Prediction : 0,Prediction : 1
0,141617,0,Cash loans,F,N,Y,0,135000.0,808650.0,23773.5,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.569481,0.430519
1,263512,0,Cash loans,M,N,Y,0,135000.0,254700.0,14751.0,...,0,0,0.0,0.0,0.0,2.0,0.0,2.0,0.894743,0.105257
2,397087,0,Revolving loans,F,Y,Y,0,234000.0,720000.0,36000.0,...,0,0,0.0,0.0,0.0,0.0,2.0,0.0,0.777016,0.222984
3,316730,1,Cash loans,F,N,N,2,67500.0,545040.0,25407.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.078817,0.921183
4,441436,0,Cash loans,F,N,Y,1,225000.0,817560.0,29493.0,...,0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.897290,0.102710


In [34]:
df.shape

(184504, 124)

In [33]:
df = df[df["CODE_GENDER"] != "XNA"]

In [35]:
df.to_csv("app_train_dashboard_with_prediction", index=False)

In [36]:
df.groupby(by="CODE_GENDER")["Prediction : 0", "Prediction : 1"].mean()

C:\Users\tothm\AppData\Local\Temp\ipykernel_14656\1448892206.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby(by="CODE_GENDER")["Prediction : 0", "Prediction : 1"].mean()


,Prediction : 0,Prediction : 1
CODE_GENDER,,
F,0.631791,0.368209
M,0.513180,0.486820
